# Estimation of covariance matrices

In [25]:
import math
import numpy as np
import matplotlib.pyplot as plt

from covariance.analytical_shrinkage import AnalyticalShrinkage
from covariance.linear_shrinkage import LinearShrinkage
from covariance.estimator import SampleEstimator, FSOptEstimator, EmpiricalBayesianEstimator
from covariance.metrics import loss_mv, loss_frobenius, PRIAL_mv

In [66]:
analshr = AnalyticalShrinkage()
linshr = LinearShrinkage()
sample_est = SampleEstimator()
eb_est = EmpiricalBayesianEstimator

## Sampling random population covariance matrix (Sigma)

#### First, let's build a method to sample orthogonal random matrices

In [10]:
def sample_rand_orthogonal_mtx(n):
    # n by n random complex matrix
    X = np.random.randn(n,n)
    # orthonormalizing matrix using QR algorithm
    Q,_ = np.linalg.qr(X)
    return Q

In [18]:
M = sample_rand_orthogonal_mtx(5) 
np.round(np.matmul(M, M.T), 12)

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -0., -0.],
       [ 0.,  0., -0.,  1.,  0.],
       [ 0.,  0., -0.,  0.,  1.]])

#### Now, let's sample a diagonal matrix with a certain proportion of eigenvalues

In [36]:
def sample_diagEig_mtx(p):
    ONE_PROP = 0.2    # proportion of eigenvalues equal to one
    THREE_PROP = 0.4  # proportion of eigenvalues equal to three
    TEN_PROP = 0.4    # proportion of eigenvalues equal to ten

    n_one = math.ceil(p * ONE_PROP)       # number of eigenvalues equal to one
    n_three = math.floor(p * THREE_PROP)  # number of eigenvalues equal to three
    n_ten = p - n_one - n_three           # number of eigenvalues equal to ten

    # building eigenvalues
    one_eigs = [1.0]*n_one
    three_eigs = [3.0]*n_three
    ten_eigs = [10.0]*n_ten
    # concatenating eigenvalues lists
    eigs = one_eigs + three_eigs + ten_eigs
    # shuffling eigenvalues
    np.random.shuffle(eigs)
    # building diagonal matrix
    M = np.diag(eigs)
    return M

In [45]:
sample_diagEig_mtx(5)

array([[ 3.,  0.,  0.,  0.,  0.],
       [ 0., 10.,  0.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0., 10.]])

#### to create a random population matrix (not necessarily diagonal) we use both functions

In [51]:
def sample_pop_cov(p, diag=False):
    if diag:
        return sample_diagEig_mtx(p)
    else:
        O = sample_rand_orthogonal_mtx(p)
        M = sample_diagEig_mtx(p)
        # O M O.T preserves original eigenvalues (O is an orthogonal rotation)
        return np.matmul(np.matmul(O, M), O.T) # sampling \Sigma

In [53]:
Sigma = sample_pop_cov(5)
np.linalg.eigvals(Sigma)

array([ 1., 10., 10.,  3.,  3.])

## Sampling dataset using population covariance matrix (Sigma)

In [62]:
p, n = 5, 10

Sigma = sample_pop_cov(p)
X = np.random.multivariate_normal(np.random.randn(p), Sigma, size=n)
print('Shape:', X.shape)
print('Random generated dataset:\n', X)

Shape: (10, 5)
Random generated dataset:
 [[-2.55854161 -0.21032277  0.34164514  1.74807977 -1.92248937]
 [ 4.05721814  0.57437241 -1.80716247 -3.64794955  4.3723337 ]
 [-3.49631905  0.51208678 -2.36383886 -0.04511112  2.13134723]
 [-5.39668058 -2.34351835 -0.19474646  2.27195219 -2.28249785]
 [-5.17591679 -0.56141896  3.13210634  2.92909646 -1.47376948]
 [-3.07709575 -1.3481458   1.1436639   0.45434139 -1.44333464]
 [ 0.99716777 -0.60400778 -1.24983323  1.133208    2.45073637]
 [ 1.51870019 -0.53705     3.01233139 -0.74123213  1.68977933]
 [-4.57658339 -1.42730276 -0.28265274  1.26742946  1.54039592]
 [-2.68474052  5.64446152  1.1995579   0.24871691  3.17230273]]


## Testing estimators and metrics

In [64]:
p, n = 200, 600

Sigma = sample_pop_cov(p)
X = np.random.multivariate_normal(np.random.randn(p), Sigma, size=n)
print('Shape:', X.shape)

Shape: (600, 200)


In [73]:
Sample = sample_est.estimate(X)
print('S sample estim. shape:', Sample.shape)

S sample estim. shape: (200, 200)


In [69]:
fsopt_est = FSOptEstimator(Sigma)
S_star = fsopt_est.estimate(X)
print('S_star shape:', S_star.shape)

S_star shape: (200, 200)


### Checking that PRIAL(Sample) = 0%

In [79]:
E_Sn = loss_mv(Sigma_tilde=Sample, Sigma=Sigma)
E_Sigma_tilde = loss_mv(Sigma_tilde=Sample, Sigma=Sigma)
E_Sstar = loss_mv(Sigma_tilde=S_star, Sigma=Sigma)

PRIAL_mv(E_Sn=E_Sn, E_Sigma_tilde=E_Sigma_tilde, E_Sstar=E_Sstar)

0.0

### Checking that PRIAL(S_star) = 100%

In [77]:
E_Sn = loss_mv(Sigma_tilde=Sample, Sigma=Sigma)
E_Sigma_tilde = loss_mv(Sigma_tilde=S_star, Sigma=Sigma)
E_Sstar = loss_mv(Sigma_tilde=S_star, Sigma=Sigma)

PRIAL_mv(E_Sn=E_Sn, E_Sigma_tilde=E_Sigma_tilde, E_Sstar=E_Sstar)

1.0

#### After these tests we are more confident that estimators FSOptEstimator and SampleEstimator, as well as metrics loss_mv and PRIAL_mv, are working properly

# Monte Carlo simulations

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

from covariance.analytical_shrinkage import AnalyticalShrinkage
from covariance.linear_shrinkage import LinearShrinkage
from covariance.estimator import SampleEstimator, FSOptEstimator, EmpiricalBayesianEstimator
from covariance.metrics import loss_mv, loss_frobenius, PRIAL_mv

In [81]:
def sample_rand_orthogonal_mtx(n):
    # n by n random complex matrix
    X = np.random.randn(n,n)
    # orthonormalizing matrix using QR algorithm
    Q,_ = np.linalg.qr(X)
    return Q

In [ ]:
def sample_diagEig_mtx(p):
    ONE_PROP = 0.2    # proportion of eigenvalues equal to one
    THREE_PROP = 0.4  # proportion of eigenvalues equal to three
    TEN_PROP = 0.4    # proportion of eigenvalues equal to ten

    n_one = math.ceil(p * ONE_PROP)       # number of eigenvalues equal to one
    n_three = math.floor(p * THREE_PROP)  # number of eigenvalues equal to three
    n_ten = p - n_one - n_three           # number of eigenvalues equal to ten

    # building eigenvalues
    one_eigs = [1.0]*n_one
    three_eigs = [3.0]*n_three
    ten_eigs = [10.0]*n_ten
    # concatenating eigenvalues lists
    eigs = one_eigs + three_eigs + ten_eigs
    # shuffling eigenvalues
    np.random.shuffle(eigs)
    # building diagonal matrix
    M = np.diag(eigs)
    return M

In [82]:
def sample_pop_cov(p, diag=False):
    if diag:
        return sample_diagEig_mtx(p)
    else:
        O = sample_rand_orthogonal_mtx(p)
        M = sample_diagEig_mtx(p)
        # O M O.T preserves original eigenvalues (O is an orthogonal rotation)
        return np.matmul(np.matmul(O, M), O.T) # sampling \Sigma

In [83]:
def sample_dataset(p, n):
    Sigma = sample_pop_cov(p)
    X = np.random.multivariate_normal(np.random.randn(p), Sigma, size=n)
    return X, Sigma

In [85]:
P, N = 200, 600 

X, Sigma = sample_dataset(p=P, n=N)

print('Dataset size:', X.shape)
print('Sigma shape:', Sigma.shape)

Dataset size: (600, 200)
Sigma shape: (200, 200)


In [86]:
analshr = AnalyticalShrinkage()
linshr = LinearShrinkage()
sample_est = SampleEstimator()
fsopt_est = FSOptEstimator(Sigma)
eb_est = EmpiricalBayesianEstimator()